In [1]:
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: /Users/soumikdutta/PycharmProjects/pythonAIProject/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import grpc
import logging
import subprocess
import com_sixt_service_genai_connector.genai_connector_pb2 as genai_connector_pb2
import com_sixt_service_genai_connector.genai_connector_pb2_grpc as genai_connector_pb2_grpc
from tenacity import retry, stop_after_attempt, wait_random_exponential

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
GRPC_HOST = "grpc-stage.goorange.sixt.com:443"


def get_access_token():
    try:
        token_command = os.path.expanduser("~/go/bin/com.sixt.tool.get-token")

        result = subprocess.run(
            [token_command, "--env=stage"],
            capture_output=True,
            text=True,
            check=True
        )
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to get access token: {e}")
        raise

ModuleNotFoundError: No module named 'grpc'

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(4))
def send_chat_request(prompt, temperature=0, max_tokens=200, model=None):
    if model is None:
        model = "goorange-engineers.gpt4o"
    access_token = get_access_token()
    messages = [
        genai_connector_pb2.Messages(
            role="user",
            content=prompt
        )
    ]
    request = genai_connector_pb2.ChatRequest(
        temperature=temperature,
        max_tokens=max_tokens,
        messages=messages,
        model=model
    )
    metadata = [('authorization', f'Bearer {access_token}')]
    try:
        response = stub.Chat(request, metadata=metadata)
        return response
    except Exception as e:
        logger.info(f"An error occurred: {str(e)}")
        raise

Let's define gRPC channel and stub for communication with the GenAI service.

In [9]:
channel = grpc.secure_channel(GRPC_HOST, grpc.ssl_channel_credentials())
stub = genai_connector_pb2_grpc.GenAIConnectorStub(channel)

Now, we have the required setup to send chat request. 

In [10]:
prompts = [
    'case1: give me germany capital',
    "case2: give me italy capital"
]
for prompt in prompts:
    response = send_chat_request(prompt)
    print(response.choices[0].message.content)


The capital of Germany is Berlin.
The capital of Italy is Rome.


In [11]:
channel.close()